In [ ]:
import numpy as np

# Define the Rastrigin function (objective function to minimize)
def rastrigin(x):
    A = 10
    return A * len(x) + sum(x**2 - A * np.cos(2 * np.pi * x))

# Levy Flight function (helps with exploration)
def levy_flight(Lambda, d):
    sigma_u = (np.math.gamma(1 + Lambda) * np.sin(np.pi * Lambda / 2) /
               (np.math.gamma((1 + Lambda) / 2) * Lambda * 2**((Lambda - 1) / 2)))**(1 / Lambda)
    u = np.random.normal(0, sigma_u, size=d)
    v = np.random.normal(0, 1, size=d)
    step = u / np.abs(v)**(1 / Lambda)
    return step

# Cuckoo Search Algorithm
def cuckoo_search(func, dim, bounds, num_nests=25, max_iter=100, pa=0.25, alpha=0.01, beta=1.5):
    # Initialize nests randomly within bounds
    nests = np.random.uniform(bounds[0], bounds[1], (num_nests, dim))

    # Calculate fitness of each nest
    fitness = np.array([func(nest) for nest in nests])

    # Initialize the best nest and its fitness
    best_nest = nests[np.argmin(fitness)]
    best_fitness = np.min(fitness)

    # Main loop for the algorithm
    for iteration in range(max_iter):
        for i in range(num_nests):
            # Generate new nest using Levy flight
            step = alpha * levy_flight(beta, dim)
            new_nest = nests[i] + step * (nests[i] - best_nest)  # Move towards the best nest

            # Ensure the new nest is within bounds
            new_nest = np.clip(new_nest, bounds[0], bounds[1])

            # Evaluate the new nest
            new_fitness = func(new_nest)

            # If new nest is better, replace the old nest
            if new_fitness < fitness[i]:
                nests[i] = new_nest
                fitness[i] = new_fitness

        # Replace some of the worst nests
        worst_nests_idx = np.argsort(fitness)[-int(pa * num_nests):]
        nests[worst_nests_idx] = np.random.uniform(bounds[0], bounds[1], (len(worst_nests_idx), dim))
        fitness[worst_nests_idx] = np.array([func(nest) for nest in nests[worst_nests_idx]])

        # Update the best solution
        min_idx = np.argmin(fitness)
        if fitness[min_idx] < best_fitness:
            best_fitness = fitness[min_idx]
            best_nest = nests[min_idx]

        # Print the best fitness every 10 iterations for less verbose output
        if iteration % 10 == 0 or iteration == max_iter - 1:
            print(f"Iteration {iteration + 1}/{max_iter}, Best Fitness: {best_fitness}")

    return best_nest, best_fitness

# Define the problem bounds and parameters
dim = 5  # Problem dimensionality (5-dimensional optimization)
bounds = [-5.12, 5.12]  # Bounds for the search space (for Rastrigin function)
num_nests = 25  # Number of nests (solutions) to consider
max_iter = 50  # Reduce the number of iterations for faster results
pa = 0.25  # Probability of discovering a nest (fraction of worst nests to abandon)

# Run the Cuckoo Search algorithm
best_solution, best_fitness = cuckoo_search(rastrigin, dim, bounds, num_nests, max_iter, pa)

# Output the best solution and fitness
print("\nBest solution found: ", best_solution)
print("Best fitness (objective value): ", best_fitness)


Iteration 1/50, Best Fitness: 54.99284923697366
Iteration 11/50, Best Fitness: 37.967575157273394
Iteration 21/50, Best Fitness: 30.36128586284407
Iteration 31/50, Best Fitness: 23.259817673200633
Iteration 41/50, Best Fitness: 19.234263008468304
Iteration 50/50, Best Fitness: 19.234263008468304

Best solution found:  [ 0.00587978 -1.88561194  0.86389239 -0.87520271  1.90599959]
Best fitness (objective value):  19.234263008468304


<ipython-input-3-fb4a1a18f5fb>:10: DeprecationWarning: `np.math` is a deprecated alias for the standard library `math` module (Deprecated Numpy 1.25). Replace usages of `np.math` with `math`
  sigma_u = (np.math.gamma(1 + Lambda) * np.sin(np.pi * Lambda / 2) /
<ipython-input-3-fb4a1a18f5fb>:11: DeprecationWarning: `np.math` is a deprecated alias for the standard library `math` module (Deprecated Numpy 1.25). Replace usages of `np.math` with `math`
  (np.math.gamma((1 + Lambda) / 2) * Lambda * 2**((Lambda - 1) / 2)))**(1 / Lambda)
